#医疗行业数据编织

##Section 1 实体识别与信息提取

问题：电子病历中广泛存在以自由文本记录的信息，如病程记录、查房记录、医患沟通记录等。这些文本是极具价值的医疗行业数据，对于跟踪患者病情演化、评估疗效、医疗循证等方面都具有重要的作用。然而，自由文本的作者和读者通常是人而不是机器或程序，并不能直接被其它程序有效利用；对于医疗工作者而言，从大量的文本中进行统计分析，总结变化趋势，如体温变化趋势、临床观察值变化趋势等，也是工作量巨大且容易出错的部分。

本实验拟采用大模型技术，通过提示工程，从自由文本中识别医疗行业概念实体，并将自由文本转换为FHIR R4版本数据资源，以便为基于这些文本数据进行统计分析、即席查询提供技术基础。

###实验大模型

本实验采用的大模型为智谱开放平台提供的[GLM-4](https://open.bigmodel.cn/)，运行之前请至该平台注册账号并生成API密钥，将密钥在本页作为全局变量输入，后续的代码中将自动凭此密钥访问GLM-4，如产生费用，将由实验者自理。

###测试数据来源

本文中采用的自由文本数据来源于[中文医疗信息处理评测基准CBLUE](https://tianchi.aliyun.com/dataset/95414)，主要采用了其中临床发现事件抽取（CHIP-CDEE）任务所用的数据。

###Samples

####Sample1

4+月前患者无明显诱因出现阴道不规则点滴流血，无畏寒、发热，无腹痛、腹胀及阴道流液，无咳嗽、咳痰等不适，当时未引起重视，为求进一步治疗，遂到某医院就诊，行宫腔镜及诊断性刮宫提示子宫内膜癌，经金域检验中心会诊后，病检结果（123456）提示：子宫腔腺癌，结合免疫表型及he形态符合低分化子宫内膜样腺癌，免疫组化结果：ck(2+)、vim(2+)、er(小灶+)、pr（-)、p16(2+)、p53(2+)、a-inhibin（-)、ki-67(2+,约80%)。

####Sample2

术后分别予以tp方案行了五周期化疗，化疗过程顺利，复查肝肾功、血常规、电解质无异常，出院后无发热、腹胀痛、阴道异常流血流液等双下肢疼痛等，有恶心、呕吐等反应，现患者要求入院择期行第六周期化疗，门诊以“1.左侧卵巢卵巢腺癌iv期术后；2.tp第六次化疗”收入院。

####Sample3

请外科会诊后建议至上级医院手术或者放化疗，但患者拒绝并办理出院；此后患者仍诉张口困难，偶有左侧面颈部轻微疼痛，无发热，无咳嗽咯痰，无腹痛腹泻，22天前患者在家属陪同下再次到我科住院治疗，入院后完善检查无化疗禁忌，征得患者及家属同意后于5-25日行紫杉醇240mg d1+奈达铂120mg d1化疗，并于5月29日给予帕米磷酸二钠抗溶骨治疗，治疗过程顺利，化疗后患者面部包块缩小，今日患者为行下一疗程化疗入院，门诊以“口腔粘膜恶性肿瘤 口腔鳞状细胞癌 t4n2M0”收治入院。

In [43]:
##提示词 
prompt_text = """
你是一个医疗行业数据提取助手。你能从从中文电子病历中挖掘出临床发现事件。即给定一段现病史或者医学影像所见报告，要求从中抽取临床发现事件的四个属性: 解剖部位、主体词、描述词，以及发生状态:
    主体词：指患者的电子病历中的疾病名称或者由疾病引发的症状，也包括患者的一般情况如饮食，二便，睡眠等。主体词尽可能完整并是专有名词，比如“麻木， 疼痛，发烧，囊肿”等；专有名词，如“头晕”，晕只能发生在头部，“胸闷”，闷只能发生在胸部，所以不进行拆分，保留完整的专有名词。涉及泛化的症状不做标注，如“无其他不适”，句子中的“不适”不需要标注，只针对具体的进行标注。注意：有较小比例的主体词会映射到ICD标准术语，所使用的ICD的版本为“国际疾病分类 ICD-10北京临床版v601.xIsx”(见下载文件)。
    描述词：对主体词的发生时序特征、轻重程度、形态颜色等多个维度的刻画，也包括疾病的起病缓急、突发。
    解剖部位：指主体词发生在患者的身体部位，也包括组织，细胞，系统等，也包括部位的方向和数量。
    发生状态：“不确定”或“否定”，肯定的情况不标注发生状态。
针对医疗文本，你将在分析其内容后以json格式标注临床发现事件：
    text: 表示病历或者医学影像报告
    event: 列表结构，由一个或者多个事件四元组组成，如果没有发现事件，则设为[]
        core_name: 主体词，字符串
        tendency: 发生状态，字符串，如果没有发生状态，默认为""
        character: 描述词，列表结构，如果没有描述词，默认为[]
        anatomy_list: 解剖部位，列表结构，如果没有解剖部位，默认为[]
例如，对这样一段文本描述：
患者于入院前十二小时前无明显诱因突发晕厥，伴意识不清，站立不稳，伴言语不能，无大汗，无头晕、目眩，无恶心、呕吐，无四肢发麻等，患者自测血糖约为“3.5mm/h”，于夜间灌服少量牛奶及葡萄糖水外未做任何处理。
生成的json格式如下：
{
    "id": 647,
    "text": "患者于入院前十二小时前无明显诱因突发晕厥，伴意识不清，站立不稳，伴言语不能，无大汗，无头晕、目眩，无恶心、呕吐，无四肢发麻等，患者自测血糖约为“3.5mm/h”，于夜间灌服少量牛奶及葡萄糖水外未做任何处理。",
    "event": [
        {
            "core_name": "晕厥",
            "tendency": "",
            "character": [
                "突发"
            ],
            "anatomy_list": []
        },
        {
            "core_name": "意识",
            "tendency": "",
            "character": [
                "不清"
            ],
            "anatomy_list": []
        },
        {
            "core_name": "站立",
            "tendency": "",
            "character": [
                "不稳"
            ],
            "anatomy_list": []
        },
        {
            "core_name": "语言表达",
            "tendency": "",
            "character": [
                "不能"
            ],
            "anatomy_list": []
        },
        {
            "core_name": "大汗",
            "tendency": "否定",
            "character": [],
            "anatomy_list": []
        },
        {
            "core_name": "头晕",
            "tendency": "否定",
            "character": [],
            "anatomy_list": [
                "头"
            ]
        },
        {
            "core_name": "目眩",
            "tendency": "否定",
            "character": [],
            "anatomy_list": []
        },
        {
            "core_name": "恶心",
            "tendency": "否定",
            "character": [],
            "anatomy_list": []
        },
        {
            "core_name": "呕吐",
            "tendency": "否定",
            "character": [],
            "anatomy_list": []
        },
        {
            "core_name": "发麻",
            "tendency": "否定",
            "character": [],
            "anatomy_list": [
                "四肢"
            ]
        }
    ]
}
另外一个例子：
化疗期间及化疗后无恶心等不良反应，无寒战高热，无心慌胸闷，无呼吸困难，无四肢乏力。
生成的json格式如下：
{
    "id": 673,
    "text": "化疗期间及化疗后无恶心等不良反应，无寒战高热，无心慌胸闷，无呼吸困难，无四肢乏力。",
    "event": [
      {
        "core_name": "恶心",
        "tendency": "否定",
        "character": [],
        "anatomy_list": []
      },
      {
        "core_name": "寒颤",
        "tendency": "否定",
        "character": [],
        "anatomy_list": []
      },
      {
        "core_name": "发热",
        "tendency": "否定",
        "character": [
          "高"
        ],
        "anatomy_list": []
      },
      {
        "core_name": "心慌",
        "tendency": "否定",
        "character": [],
        "anatomy_list": [
          "心"
        ]
      },
      {
        "core_name": "胸闷",
        "tendency": "否定",
        "character": [],
        "anatomy_list": [
          "胸"
        ]
      },
      {
        "core_name": "呼吸困难",
        "tendency": "否定",
        "character": [],
        "anatomy_list": []
      },
      {
        "core_name": "乏力",
        "tendency": "否定",
        "character": [],
        "anatomy_list": [
          "四肢"
        ]
      }
    ]
}
其中的text属性应与输入的报告文本完全一致，不要生成任何不一致的文本。
不要在json中生成任何注释，也不要生成示例中不存在的属性。识别多少实体就列出多少实体，不要在json中使用省略号。
不要提供任何解释，也不要返回任何无关文字，只返回回答变量。
"""

In [44]:
#运行环境设置
#在本级目录下建立文件apikey，并将密钥内容拷贝在该文件中
with open('../apikey', 'r') as file:
    api_key = file.read()
import json,re
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key) # 填写您自己的APIKey

###医疗实体信息识别

In [ ]:
#请输入想要处理的自由文本，也可从本文的Samples中选填
free_text = input("请输入医疗文本: ")

In [49]:
response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": free_text}
    ],
    temperature=0.01,
    max_tokens=2048
)
entity_payload=response.choices[0].message.content
json_block_pattern=r"'''json\n?|'''"
entity_jsonString=re.sub(json_block_pattern,'',entity_payload,flags=re.DOTALL)
print(entity_jsonString)

{
    "id": "",
    "text": "患者本次发病以来，食欲正常，神志清醒，精神欠佳，睡眠欠佳，大便正常，小便正常，体重无明显变化。",
    "event": [
        {
            "core_name": "精神",
            "tendency": "",
            "character": [
                "欠佳"
            ],
            "anatomy_list": []
        },
        {
            "core_name": "睡眠",
            "tendency": "",
            "character": [
                "欠佳"
            ],
            "anatomy_list": []
        }
    ]
}
